<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
SVR for Regression 
</p>

In [ ]:
# Libraries for data manipulation and visualization
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Modeling: SVR + pipeline + CV + metrics
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import PredefinedSplit, learning_curve
from sklearn.metrics import (
    mean_squared_error, r2_score, mean_absolute_percentage_error, median_absolute_error
)

# Bayesian optimization (same tool as RF/KNN)
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

# Utility
from scipy.stats import linregress
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='skopt')
warnings.filterwarnings("ignore", category=FutureWarning)

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Load the same standardized dataset and folds
</p>

In [ ]:
# Path to the standardized database directory 
base_path = '../../Comprehensive ML - Files & Plots etc'

# Load train and test splits
df_train = pd.read_csv(f"{base_path}/train.csv")
df_test  = pd.read_csv(f"{base_path}/test.csv")

feature_names = [
    'distance', 'frequency', 'c_walls', 'w_walls', 'co2', 'humidity', 
    'pm25', 'pressure', 'temperature', 'snr'
]

X_train = df_train[feature_names].values
y_train = df_train['PL'].values
X_test  = df_test[feature_names].values
y_test  = df_test['PL'].values

# (Should we need them for plotting)
time_train = df_train['time'].values
time_test  = df_test['time'].values

# Load 5-fold assignments (PredefinedSplit)
fold_assignments = np.load(f"{base_path}/train_folds.npy")

print(f"\nTraining samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")
unique, counts = np.unique(fold_assignments, return_counts=True)
print("Fold sizes:", dict(zip(unique, counts)))
print('\nDataset loaded successfully!\n')

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Define SVR pipeline + search space
</p>

#### Why a pipeline? SVR is highly scale‑sensitive (C and ε depend on the target scale and feature magnitudes). We standardize features inside CV. I also let the search toggle the scaler (Standard vs Robust) to hedge against outliers.

In [ ]:
# ---- Multi-metric scoring (same as RF/KNN) ----
scoring = {
    'neg_root_mean_squared_error': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

# ---- PredefinedSplit ----
ps = PredefinedSplit(fold_assignments)

# ---- Bayesian optimization ----
bayes_cv_svr = BayesSearchCV(
    estimator=create_svr_pipeline(),
    search_spaces=search_spaces,
    n_iter=30,                               # adjust for runtime
    scoring=scoring,
    refit='neg_root_mean_squared_error',     # choose best by RMSE
    n_jobs=-1,
    cv=ps,
    random_state=42,
    verbose=2,
    n_points=1,
    optimizer_kwargs={'n_initial_points': 12, 'acq_func': 'gp_hedge'}
)

print(f"Starting Bayesian optimization with {bayes_cv_svr.n_iter} iterations "
      f"and {ps.get_n_splits()}-fold cross-validation per candidate...")

bayes_cv_svr.fit(X_train, y_train)

print("Bayesian optimization complete. Extracting results...")

# ---- Pull all tried configs/results into a dataframe ----
bayes_results_svr = pd.DataFrame(bayes_cv_svr.cv_results_).copy()
print(f"Tried {bayes_results_svr.shape[0]} SVR configurations.")

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Bayesian optimization with PredefinedSplit + two metrics
</p>

In [ ]:
# ---- Multi-metric scoring (same as RF/KNN) ----
scoring = {
    'neg_root_mean_squared_error': 'neg_root_mean_squared_error',
    'r2': 'r2'
}

# ---- PredefinedSplit ----
ps = PredefinedSplit(fold_assignments)

# ---- Bayesian optimization ----
bayes_cv_svr = BayesSearchCV(
    estimator=create_svr_pipeline(),
    search_spaces=search_spaces,
    n_iter=30,                               # adjust for runtime
    scoring=scoring,
    refit='neg_root_mean_squared_error',     # choose best by RMSE
    n_jobs=-1,
    cv=ps,
    random_state=42,
    verbose=2,
    n_points=1,
    optimizer_kwargs={'n_initial_points': 12, 'acq_func': 'gp_hedge'}
)

print(f"Starting Bayesian optimization with {bayes_cv_svr.n_iter} iterations "
      f"and {ps.get_n_splits()}-fold cross-validation per candidate...")

bayes_cv_svr.fit(X_train, y_train)

print("Bayesian optimization complete. Extracting results...")

# ---- Pull all tried configs/results into a dataframe ----
bayes_results_svr = pd.DataFrame(bayes_cv_svr.cv_results_).copy()
print(f"Tried {bayes_results_svr.shape[0]} SVR configurations.")

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Summarize best CV results per kernel (for plotting & reporting)
</p>

In [ ]:
cv_summary_per_kernel = []
if 'param_svr__kernel' not in bayes_results_svr.columns:
    raise KeyError("BayesSearchCV results do not include 'param_svr__kernel'.")

for kernel in sorted(bayes_results_svr['param_svr__kernel'].dropna().astype(str).unique()):
    df_k = bayes_results_svr[bayes_results_svr['param_svr__kernel'].astype(str) == kernel]
    if not df_k.empty:
        idx = df_k['mean_test_neg_root_mean_squared_error'].idxmax()  # maximize neg RMSE
        row = df_k.loc[idx]
        best_rmse = -row['mean_test_neg_root_mean_squared_error']
        std_rmse  =  row['std_test_neg_root_mean_squared_error']
        best_r2   =  row['mean_test_r2']
        std_r2    =  row['std_test_r2']
        best_params = {c.split('param_')[1]: row[c] for c in df_k.columns if c.startswith('param_')}
        print(f"Kernel={kernel:>6s} | Best CV RMSE: {best_rmse:.4f} | R²: {best_r2:.4f} | params: {best_params}")
        cv_summary_per_kernel.append({
            'kernel': kernel,
            'best_cv_rmse': best_rmse,
            'std_cv_rmse': std_rmse,
            'best_cv_r2': best_r2,
            'std_cv_r2': std_r2,
            'best_params': best_params
        })

cv_svr_df = pd.DataFrame(cv_summary_per_kernel).sort_values('kernel').reset_index(drop=True)

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
 Plot: best CV RMSE and its STD per kernel (saved)
</p>

In [ ]:
# ---- FONT SIZE METRICS ----
tick_fontsize = 12
axis_labelsize = 15
legend_fontsize = 12

# ---- GLOBAL FONT FAMILY: Times New Roman ----
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'

sns.set_style("darkgrid")

x = np.arange(len(cv_svr_df['kernel']))
bar_width = 0.35

fig, ax1 = plt.subplots(figsize=(7, 3.5))

# Blue bars: Best CV RMSE (left y-axis)
bars1 = ax1.bar(
    x - bar_width/2, 
    cv_svr_df['best_cv_rmse'], 
    bar_width, 
    color='#1976d2',
    label='RMSE',
    edgecolor='black',
    linewidth=1,
    zorder=3
)
ax1.set_xlabel('kernel', fontsize=axis_labelsize)
ax1.set_xticks(x)
ax1.set_xticklabels(cv_svr_df['kernel'], fontsize=tick_fontsize)
ax1.tick_params(axis='y', labelcolor='#1976d2', labelsize=tick_fontsize)
ax1.grid(True, axis='y')

# Magenta bars: STD of CV RMSE (right y-axis)
ax2 = ax1.twinx()
bars2 = ax2.bar(
    x + bar_width/2, 
    cv_svr_df['std_cv_rmse'], 
    bar_width, 
    color='#d81b60',
    label='Std. of RMSE',
    edgecolor='black',
    linewidth=1,
    zorder=3
)
ax2.tick_params(axis='y', labelcolor='#d81b60', labelsize=tick_fontsize)
ax2.grid(False)

# Single legend
handles = [
    plt.Rectangle((0,0),1,1,color='#1976d2',ec='black',label='RMSE (dB)'),
    plt.Rectangle((0,0),1,1,color='#d81b60',ec='black',label='Std. of RMSE (dB)')
]
ax1.legend(handles=handles, loc='upper right', fontsize=legend_fontsize)

fig.tight_layout()
plt.savefig('../../Comprehensive ML - Files & Plots etc/SVR_bestRMSE_&_STD_perKernel.png',
            dpi=2000, bbox_inches='tight')
plt.show()

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Plot: best CV R² and its STD per kernel (saved) 
</p>

In [ ]:
# ---- FONT SIZE METRICS ----
tick_fontsize = 12
axis_labelsize = 15
legend_fontsize = 12

# ---- GLOBAL FONT FAMILY: Times New Roman ----
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'

sns.set_style("darkgrid")

x = np.arange(len(cv_svr_df['kernel']))
bar_width = 0.35

fig, ax1 = plt.subplots(figsize=(7, 3.5))

# Cyan bars: Best CV R² (left y-axis)
bars1 = ax1.bar(
    x - bar_width/2, 
    cv_svr_df['best_cv_r2'], 
    bar_width, 
    color='#00bcd4',
    edgecolor='black',
    linewidth=1,
    alpha=0.8,
    label='R$^2$'
)
ax1.set_xlabel('kernel', fontsize=axis_labelsize)
ax1.set_xticks(x)
ax1.set_xticklabels(cv_svr_df['kernel'], fontsize=tick_fontsize)
ax1.tick_params(axis='y', labelcolor='#00bcd4', labelsize=tick_fontsize)
ax1.set_ylim(0, 1.01)
ax1.grid(True, axis='y')

# Green bars: STD of CV R² (right y-axis)
ax2 = ax1.twinx()
bars2 = ax2.bar(
    x + bar_width/2, 
    cv_svr_df['std_cv_r2'], 
    bar_width, 
    color='#388e3c',
    edgecolor='black',
    linewidth=1,
    alpha=0.8,
    label='Std. of R$^2$'
)
ax2.tick_params(axis='y', labelcolor='#388e3c', labelsize=tick_fontsize)
ax2.grid(False)

# Combined Legend
handles = [
    plt.Rectangle((0,0),1,1,color='#00bcd4',ec='black',label='R$^2$'),
    plt.Rectangle((0,0),1,1,color='#388e3c',ec='black',label='Std. of R$^2$')
]
ax1.legend(handles=handles, loc='upper right', fontsize=legend_fontsize)

fig.tight_layout()
plt.savefig('../../Comprehensive ML - Files & Plots etc/SVR_bestR2_STD_perKernel.png',
            dpi=2000, bbox_inches='tight')
plt.show()

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Evaluate best SVR on train/test; save model 
</p>

In [ ]:
# Use the built-in best_estimator_ (already refitted on full train data) and best_params_
best_svr_model  = bayes_cv_svr.best_estimator_
best_svr_params = bayes_cv_svr.best_params_
print("Best SVR Parameters Found:", best_svr_params)

print("\nUsing best SVR model from BayesSearchCV (already trained on all data)...")

y_train_pred = best_svr_model.predict(X_train)
y_test_pred  = best_svr_model.predict(X_test)

train_mse   = mean_squared_error(y_train, y_train_pred)
test_mse    = mean_squared_error(y_test, y_test_pred)
train_r2    = r2_score(y_train, y_train_pred)
test_r2     = r2_score(y_test, y_test_pred)
test_rmse   = np.sqrt(test_mse)
test_mape   = mean_absolute_percentage_error(y_test, y_test_pred)
test_med_ae = median_absolute_error(y_test, y_test_pred)

results = pd.DataFrame({
    'Metric': [
        'Training Loss (MSE)', 'Test Loss (MSE)', 'Test RMSE',
        'R² Score (Train)', 'R² Score (Test)', 'Test MAPE (%)', 'Test Median AE'
    ],
    'Value': [
        train_mse, test_mse, test_rmse, train_r2, test_r2,
        test_mape * 100, test_med_ae
    ]
})

print("\nModel Evaluation Metrics:")
display(results)

# Ensure the directory exists (align to RF/KNN saving pattern)
os.makedirs('../Models', exist_ok=True)

# Save the trained SVR model
with open('../Models/svr_final_model.pkl', 'wb') as f:
    pickle.dump(best_svr_model, f)

print("Trained SVR model saved to ../Models/svr_final_model.pkl")

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
Diagnostics: learning curve, residuals, physics, predicted vs. real (saved)
</p>

In [ ]:
svr_model = best_svr_model  # alias

figsize = (7, 3)
path = '../../Comprehensive ML - Files & Plots etc/'

# 1. Learning Curve (neg RMSE)
train_sizes, train_scores, test_scores = learning_curve(
    svr_model, X_train, y_train,
    cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5),
    scoring='neg_root_mean_squared_error'
)
train_rmse = -train_scores.mean(1)
test_rmse  = -test_scores.mean(1)

plt.figure(figsize=figsize)
plt.plot(train_sizes, train_rmse, 'o-', label='Train RMSE')
plt.plot(train_sizes, test_rmse,  'o-', label='Test RMSE')
plt.xlabel('Training Size')
plt.ylabel('RMSE (dB)')
plt.title('SVR Learning Curve')
plt.legend()
plt.savefig(f'{path}SVR_learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Residuals
y_test_pred = svr_model.predict(X_test)
residuals   = y_test - y_test_pred
plt.figure(figsize=figsize)
plt.scatter(y_test_pred, residuals, alpha=0.5)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicted PL (dB)')
plt.ylabel('Residuals (dB)')
plt.title('SVR Residuals')
plt.savefig(f'{path}SVR_residuals.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Physics Consistency (PL vs. log(distance))
dist     = df_test['distance'].values
log_dist = np.log10(dist + 1e-6)
slope, intercept, r_value, _, _ = linregress(log_dist, y_test_pred)
plt.figure(figsize=figsize)
plt.scatter(log_dist, y_test_pred, alpha=0.5)
plt.plot(log_dist, intercept + slope * log_dist, 'r--',
         label=f'n ~ {slope:.2f}, R²={r_value**2:.2f}')
plt.xlabel('log10(Distance)')
plt.ylabel('Predicted PL (dB)')
plt.title('SVR PL vs. log(Distance)')
plt.legend()
plt.savefig(f'{path}SVR_physics_consistency.png', dpi=300, bbox_inches='tight')
plt.show()

# 4. Predicted vs. Real
plt.figure(figsize=figsize)
lims = [min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())]
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot(lims, lims, 'r--')
plt.xlabel('Real PL (dB)')
plt.ylabel('Predicted PL (dB)')
plt.title('SVR Predicted vs. Real')
plt.savefig(f'{path}SVR_pred_vs_real.png', dpi=300, bbox_inches='tight')
plt.show()

<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
SVR model complexity diagnostic
</p>

In [ ]:
# For SVR, the number of support vectors indicates model complexity.
# Note: attribute exists after fitting; % of SVs close to 100% often signals overfitting.
try:
    n_sv = best_svr_model.named_steps['svr'].support_.shape[0]
    frac_sv = n_sv / X_train.shape[0]
    print(f"Support vectors: {n_sv} ({frac_sv:.2%} of training samples)")
except Exception as e:
    print("Could not compute support vector count:", e)
